In [162]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
full_X = pd.read_csv("./update_X.csv")
full_Y = pd.read_csv("./update_y.csv")

In [163]:
full_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30845 entries, 0 to 30844
Columns: 197 entries, X to Hour_48.SysBP
dtypes: float64(196), int64(1)
memory usage: 46.4 MB


In [226]:
X = full_X.drop(['X'],axis=1)
Y = full_Y['los_icu_grp'] - 1
X_trian = X.head(20000)
Y_train = Y.head(20000)
X_test = X.tail(10845)
Y_test = Y.tail(10845)
X_train_tensor = torch.tensor(X_trian.values).resize_(20000,4,49)
Y_train_tensor = torch.tensor(Y_train.values)
X_test_tensor = torch.tensor(X_test.values).resize_(10845,4,49)
Y_test_tensor = torch.tensor(Y_test.values)

In [227]:
print(X_train_tensor.size())
print(Y_train_tensor.size())
print(X_test_tensor.size())
print(Y_test_tensor.size())

torch.Size([20000, 4, 49])
torch.Size([20000])
torch.Size([10845, 4, 49])
torch.Size([10845])


In [228]:
X_train_tensor = X_train_tensor.transpose(2,1).double()
X_test_tensor = X_test_tensor.transpose(2,1).double()
print(X_test_tensor.shape)
print(X_test_tensor.shape)

torch.Size([10845, 49, 4])
torch.Size([10845, 49, 4])


In [229]:
n_iters/(len(train_dataset)/batch_size)

15.0

In [230]:
train_dataset = torch.utils.data.TensorDataset(X_train_tensor,Y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(X_test_tensor,Y_test_tensor)
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [231]:
# Create the model class
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # 49 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        x = torch.tensor(x,dtype=torch.float32)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 49, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out

In [232]:
# Instantiate Model Class
input_dim = 4
hidden_dim = 100
layer_dim = 1
output_dim = 5

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [233]:
# Loss function
criterion = nn.CrossEntropyLoss()

In [244]:
# Optimizer class
learning_rate = 0.005
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [245]:
len(list(model.parameters()))

6

In [246]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([400, 4])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([5, 100])
torch.Size([5])


In [ ]:
# Train Model

#Training 1 Hidden Layer LSTM
# Number of steps to unroll
seq_dim = 49
iter = 0
loss_ = list();
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        # Load features as a torch tensor with gradient accumulation abilities
        features = features.view(-1, seq_dim, input_dim).requires_grad_()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 5
        outputs = model(features)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for f, l in test_loader:
                # Resize images
                f = f.view(-1, seq_dim, input_dim)

                # Forward pass only to get logits/output
                outputs = model(f)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += l.size(0)

                # Total correct predictions
                correct += (predicted == l).sum()

            accuracy = 100 * correct / total
            # Print Loss
            loss_.append(loss.item())
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

//anaconda/envs/3/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Iteration: 3. Loss: 0.4906065762042999. Accuracy: 85
Iteration: 6. Loss: 0.6029531955718994. Accuracy: 85
Iteration: 9. Loss: 0.6363340020179749. Accuracy: 85
Iteration: 12. Loss: 0.528878927230835. Accuracy: 85
Iteration: 15. Loss: 0.6317803859710693. Accuracy: 85
Iteration: 18. Loss: 0.4884442090988159. Accuracy: 85
Iteration: 21. Loss: 0.5391638875007629. Accuracy: 85
Iteration: 24. Loss: 0.38003838062286377. Accuracy: 85
Iteration: 27. Loss: 0.6850119233131409. Accuracy: 85
Iteration: 30. Loss: 0.6044291853904724. Accuracy: 85
Iteration: 33. Loss: 0.6847351789474487. Accuracy: 85
Iteration: 36. Loss: 0.5203711986541748. Accuracy: 85
Iteration: 39. Loss: 0.5438717007637024. Accuracy: 85
Iteration: 42. Loss: 0.4698109030723572. Accuracy: 85
Iteration: 45. Loss: 0.6054216623306274. Accuracy: 85
Iteration: 48. Loss: 0.5050232410430908. Accuracy: 85
Iteration: 51. Loss: 0.5381388068199158. Accuracy: 85
Iteration: 54. Loss: 0.5695299506187439. Accuracy: 85
Iteration: 57. Loss: 0.49568355